In [1]:
from ipywidgets import widgets, interact, interactive
from datetime import datetime
import widgetsnbextension
from IPython.display import display
import re
import sys
from bs4 import BeautifulSoup
def makeHier(existingTag, hierList):
#    print("hierList on entry=:::" , hierList ,":::")
    
    #the outermost list package
    #if isinstance(hierList[0], list): 
        #hierList=hierList[0]
    newTag=None
    for entry in hierList:
        
#        print("for entry=", entry, "of type ",type(entry))
        if isinstance(entry, (dict,)):
                                #print("in dict:", lst)
            for attr in entry:
                existingTag[attr]=entry[attr]
        else:
            #sequential lists
            if isinstance(entry, (list,)):
                newTag=soup.new_tag(entry[0], "")
#                #print("nt=",newTag)
                existingTag.append(newTag)
#                #print("entry before", entry)
                entry.pop(0)
                #print("entry after", entry)
                makeHier(newTag, entry)
#               # print(existingTag.prettify())
            else:
                #Scalar: string value
#                print("in scalar:existingTag=",existingTag," entry=", entry)
                existingTag.append(entry)
    return

FORENAMES=["Alexander"] 
fn=widgets.Text(
    value=FORENAMES[0],
    placeholder='Type something',
    description='String:',
    disabled=False
)
print("fn`:", fn.value)
SURNAMES=["Lange", "Kjelland"]
persName_AUTHOR=["persName"]
for f in FORENAMES:
    persName_AUTHOR.append(["forname", f])
for s in SURNAMES:
    persName_AUTHOR.append(["surname", s])
container = widgets.HBox(children=[fn] )    
BIRTHYEAR="1849"
DEATHYEAR="1906"
TITLE="Garman & Worse"
YEAR="1880"
PLACE="København"
PUBLISHER="Gyldendalske Boghandels Forlag (F. Hegel & Søn)"
WIKI_BOOK="https://www.wikidata.org/wiki/Q21473314"
SEX="M"
VIAF="https://viaf.org/viaf/54187212"
WIKI_AUTHOR="https://www.wikidata.org/wiki/Q318358"
RESPRESP="ELTeC encoding"
RESPNAME="Michael Preminger"
LANG="nb"
BOKNR="001"
ELTeC_DATE=datetime.now().isoformat()[0:10]
TS_AUTHOR=' '.join(FORENAMES + SURNAMES)
S_TITLE=""
S_REFTARGET=""
S_R_B_TITLE=""
S_R_B_AUTHOR=S_R_B_AUTHOR=''.join(
                       [ f[0] + ". " for f in FORENAMES ] 
                     + [ s[0] + ". " for s in SURNAMES[0:-1] ] 
                     + [ SURNAMES[-1] ]
                    )
soup=None
with open("BS_Kielland_GarmanOgWorse_tekst.xml") as fp:
    soup = BeautifulSoup(fp,  "lxml")

for child in soup.find("titlestmt").find("title").descendants:
    if "Navigable" in str(type(child)):   
        S_TITLE+=str(child)
#print(S_TITLE)



    
#find number of words
nr_words=0
for p in soup.find_all('p'):
    words=re.split(' ', str(p.string))
    nr_words+=len(words)

nr_pages=0
for pb in soup.find_all('pb'):
    nr_pages+=1

#Behandle TEI-header
teiHeader=soup.find("teiheader")
teiHeader.clear()

teiHeader['type']="novelHeader"


teiHeader.append(soup.new_tag("fileDesc"))
print("th=")
#print(teiHeader.prettify())

fileDesc=teiHeader.find("fileDesc")

print("fileDesc before:", "fd=")
#print(fileDesc.encode(formatter="minimal"))

makeHier(fileDesc,
         [
            ["titleStmt", 
                            ["title", TITLE], 
                            ["author", persName_AUTHOR,                                         
                                       ["persName", S_R_B_AUTHOR],
                                       ["birth", {"when": BIRTHYEAR}],
                                       ["death", {"when": DEATHYEAR}],
                                       ["sex", {"value": SEX}],
                                       ["idno", {"type": "viaf"}, VIAF],
                                       ["idno", {"type": "wiki"}, WIKI_AUTHOR] 
                            
                            ],
                            ["respStmt", ["resp", "ELTeC encoding"], 
                                         ["name", "Michael Preminger"]
                            ]
            ],
            ["extent", 
                 ["measure", {"unit":"words" ,"quantity": nr_words} ],
                 ["measure", {"unit":"pages" ,"quantity": nr_pages} ]
            ]
             ,
            ["publicationStmt",
                 ["p", "Incorporated into the ELTeC", 
                      ["date", datetime.now().isoformat()[0:10]]
                 ]
            ],
            ["sourceDesc",
                ["bibl", 
                 ["author", S_R_B_AUTHOR],
                 ["title", S_TITLE],
                 ["publPlace", PLACE],
                 ["publisher", PUBLISHER],
                 ["date", YEAR],
                 ["idno", {"type":"wiki"}, WIKI_BOOK]
                ],
                ["bibl", 
                 ["title", S_TITLE],
                 ["ref", {"target":"wiki"}, "some ref"]
                ] 
            ]
            
         ]
              
        )

#print("fileDesc after:", "fd=")
#print(fileDesc.prettify(formatter="minimal"))
#print(fileDesc.prettify(formatter="xml"))


teiHeader.append(soup.new_tag("profileDesc"))
profileDesc=teiHeader.find("profileDesc")


teiHeader.insert(3, soup.new_tag("revisionDesc"))
revisionDesc=teiHeader.find("revisionDesc")


titleStatement=teiHeader.find("titleStatement")
                 
                 

#FINN OG FJERN FORMATTERING AV DIREKTETALE 
#print(len(soup.find_all('hi')))
for hi in soup.find_all('hi'):
    soup.hi.unwrap()
    
#Sjekk at alt er fjernet
print("har behandlet")
print(len(soup.find_all('hi')))

#FINN KAPITTEL-DIVer
for div in soup.find_all('div', n=re.compile("\d"), type='chapter'):
    if div.get('n') is not None :
        n=div['n']
        div['xml:id']=LANG+BOKNR+n.zfill(3)
    

#SKRIV UT BEHANDLET FIL
textFile=open("test_output.xml", "w")
textFile.write(str(soup))

textFile.close()
def response(change):
    #if validate():
    forname=fn.value
    print("ffffffff=",forname)
        


fn.observe(response, names="value")
display(container)

fn`: Alexander
th=
fileDesc before: fd=
har behandlet
0


ffffffff= Alexande
ffffffff= Alexand
ffffffff= Alexan
ffffffff= Alexa
ffffffff= Alex
ffffffff= Ale


In [ ]:
'''
,
            ["publicationStmt",
                 ["p", "Incorporated into the ELTeC", 
                      ["date", datetime.now().isoformat()[0:10]]
                 ]
            ],
            ["sourceDesc",
                ["bibl", 
                 ["author", S_R_B_AUTHOR],
                 ["title", S_TITLE],
                 ["publPlace", PLACE],
                 ["publisher", PUBLISHER],
                 ["date", YEAR],
                 ["idno", {"type":"wiki"}, WIKI_BOOK]
                ],
                ["bibl", 
                 ["title", S_TITLE],
                 ["ref", {"target":"wiki"}, "some ref"]
                ] 
            ]
         ]         

'''

In [ ]:
for hi in soup.find_all('hi'):
    print(hi.unwrap()) 

'''
hi_tag = soup.hi
hi_tag.unwrap()
'''

In [ ]:
print(len(soup.find_all('hi')))